In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
nft_price = pd.read_csv('NFT_price.csv', sep=';', decimal=',')
nft_price['Date'] = pd.to_datetime(nft_price['Date'])
nft_price.head()

,Date,SalesUSD
0,2017-06-22,0.00
1,2017-06-23,1020.30
2,2017-06-24,1240.84
3,2017-06-25,517.55
4,2017-06-26,424.63


In [5]:
eth_price = pd.read_csv('ETH-USD.csv')
eth_price['Date'] = pd.to_datetime(eth_price['Date'])
eth_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776 entries, 0 to 1775
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1776 non-null   datetime64[ns]
 1   Open      1772 non-null   float64       
 2   High      1772 non-null   float64       
 3   Low       1772 non-null   float64       
 4   Close     1772 non-null   float64       
 5   AdjClose  1772 non-null   float64       
 6   Volume    1772 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 97.2 KB


In [3]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [4]:
def auth():
    return "AAAAAAAAAAAAAAAAAAAAACzyVAEAAAAAMZnwRwus1XDVUlJfRE9TiN4uAok%3DMB6ISFHyBDbsSVMc83n1CItN27fODII0OekQMKAkkR1RVY3RrY"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [5]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "NFT lang:en"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15

In [6]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [7]:
json_response["meta"]

{'newest_id': '1377047972618338309',
 'oldest_id': '1377047919304470528',
 'result_count': 14,
 'next_token': 'b26v89c19zqg8o3fosqtycrburxchd699390pjqwx0tx9'}

In [12]:
def assemble_df(keyword, start_date, end_date, max_results = 10, wait=5):
    bearer_token = auth()
    headers = create_headers(bearer_token)
    url, params = create_url(keyword, start_time,end_time, max_results)
    json_response = connect_to_endpoint(url, headers, params)
    next_token = json_response["meta"]["next_token"]
    df = pd.json_normalize(json_response["data"])
    for i in range(5):
        time.sleep(wait)
        json_response = connect_to_endpoint(url, headers, params, next_token)
        next_token = json_response["meta"]["next_token"]
        new_data = pd.json_normalize(json_response["data"])
        print(json_response["meta"]["result_count"])
        df = pd.concat([df, new_data], axis=0)
    return df


In [13]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "NFT lang:en"
start_date = "2021-03-01T00:00:00.000Z"
end_date = "2021-03-31T00:00:00.000Z"
max_results = 15
df = assemble_df(keyword, start_date, end_date)

Endpoint Response Code: 200
Endpoint Response Code: 200
9
Endpoint Response Code: 200
10
Endpoint Response Code: 200
9
Endpoint Response Code: 200
9
Endpoint Response Code: 200
10


In [37]:
df

,created_at,id,author_id,reply_settings,conversation_id,lang,source,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,in_reply_to_user_id
0,2021-03-30T23:59:55.000Z,1377047972618338309,1376260731704860672,everyone,1377047972618338309,en,Twitter Web App,"Look what I just , &gt;9⁸@opensea! #nft #opens...",0,0,0,0,NaN,NaN
1,2021-03-30T23:59:55.000Z,1377047971938861064,1121488790080638983,everyone,1377047971938861064,en,Twitter Web App,RT @Bit_Errror: Dancing with Myself by @Bit_Er...,15,0,0,0,"[{'type': 'retweeted', 'id': '1376970402317148...",NaN
2,2021-03-30T23:59:52.000Z,1377047959553015809,608469433,everyone,1377047959553015809,en,Twitter for iPhone,RT @JohnKnopfPhotos: My first foundation NFT i...,20,0,0,0,"[{'type': 'retweeted', 'id': '1375499284401901...",NaN
3,2021-03-30T23:59:50.000Z,1377047951059587075,240197581,everyone,1376929052016132096,en,Twitter for iPhone,@NFT_us World Of Ethereum Art! Part 1. Take Yo...,0,0,1,0,"[{'type': 'replied_to', 'id': '137692905201613...",1372220379599671303
4,2021-03-30T23:59:50.000Z,1377047951026098182,1270355709973213186,everyone,1377047951026098182,en,Twitter for iPhone,RT @UpdateHLD: 📲| Louis responding to a fan ab...,552,0,0,0,"[{'type': 'retweeted', 'id': '1377018665904840...",NaN
5,2021-03-30T23:59:50.000Z,1377047949444800516,1361835632671227906,everyone,1377047949444800516,en,Twitter Web App,Another gem 💎 not to be missed on #BSC #NFT ma...,0,0,1,0,NaN,NaN
6,2021-03-30T23:59:48.000Z,1377047941303656448,1372932106859507716,everyone,1377047941303656448,en,Twitter Web App,RT @Rahim_mahtab: NEW NFT !!! A Family living ...,29,0,0,0,"[{'type': 'retweeted', 'id': '1376966190367580...",NaN
7,2021-03-30T23:59:48.000Z,1377047940762464256,189626774,everyone,1377047940762464256,en,Twitter for Android,RT @chiaropteran: Silicon Valley investors sho...,1882,0,0,0,"[{'type': 'retweeted', 'id': '1376622460704075...",NaN
8,2021-03-30T23:59:47.000Z,1377047939114237953,444468221,everyone,1377047939114237953,en,Twitter Web App,RT @chiaropteran: Silicon Valley investors sho...,1882,0,0,0,"[{'type': 'retweeted', 'id': '1376622460704075...",NaN
9,2021-03-30T23:59:47.000Z,1377047937096708102,1195883458087673856,everyone,1376536596825116675,en,Twitter for Android,@DappRadar @WAX_io @BunniesRarebit Wooo excell...,0,0,0,0,"[{'type': 'replied_to', 'id': '137653659682511...",962293079012241408
